### 1、执行脚本前，本地需要准备好E:\\oraclepy目录中相关rpm包，配置文件，安装文件等，远程有/dbhome目录，
### 2、配置主机ip，端口号，用户名，密码hostname='10.2.3.68'，port=22，username='root'，password='redhat',
### 3、确认UID 1001未被占用,确认/etc/hosts文件里面有机器名及对应ip地址，
### 4、遗留问题：rootuser.ora_rdbms(cmd3)需要单独运行

In [4]:
'''
执行脚本前，本地需要准备好E:\\oraclepy目录中相关rpm包，配置文件，安装文件等，远程有/dbhome目录，
并配置主机ip，端口号，用户名，密码hostname='10.2.3.68'，port=22，username='root'，password='redhat',UID 1001未被占用
'''
#定义函数__get_all_file_in_local_dir列出指定路径下所有文件路径+文件名
import paramiko
import os

def __get_all_file_in_local_dir(local_dir):
    all_files=list()
    files=os.listdir(local_dir)
    for x in files:
        filename=os.path.join(local_dir,x)
        if os.path.isdir(x):
            all_files.extend(__get_all_file_in_local_dir(filename))
        else:
            all_files.append(filename)
    return all_files

#定义函数sftp_put_dir将本地路径通过sftp传给对应服务器路径
def sftp_put_dir(local_dir,remote_dir):
    t=paramiko.Transport((hostname,port))
    t.connect(username=username,password=password)
    sftp=paramiko.SFTPClient.from_transport(t)
    if remote_dir[-1]=='/':
        remote_dir=remote_dir[0:-1]
    all_files=__get_all_file_in_local_dir(local_dir)
    for x in all_files:
        filename=os.path.split(x)[-1]
        remote_filename=remote_dir+'/'+filename
        print (u'Putting file %s...' % filename)
        sftp.put(x,remote_filename)
    t.close()

#传输所需rpm包给服务器
#if __name__=="__main__":
hostname='10.2.3.233'
port=22
username='root'
password='redhat'
'''
上面四个动态配置参数
'''
local_dir=r'E:\\oraclepy\\oraclerpm'
remote_dir=r'/dbhome'
sftp_put_dir(local_dir,remote_dir)
print ('upload rpm files successful!' )

#创建类Linux_rpm实现安装rpm包并检查rpm记录日志
class Linux_rpm():
    def __init__(self,hostname,username,password):
        self.hostname=hostname
        self.username=username
        self.password=password
    def rpm_install(self,__cmd):
        try:
            paramiko.util.log_to_file('root_rpm.log')
            s=paramiko.SSHClient()
            s.load_system_host_keys()
            s.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            s.connect(self.hostname,22,self.username,self.password)
            stdin,stdout,stderr=s.exec_command(__cmd)
            stdout.read()
        except Exception as e:
            print ('error',e)
            s.close()
        finally:
            s.close()

#安装rpm包并检查rpm记录日志
#if __name__=="__main__":
cmd="cd /dbhome;sh ora_rpm_install.sh>ora_rpm_install.txt"
host=Linux_rpm(hostname,username,password)
host.rpm_install(cmd)
print (' run ora_rpm_install.sh complete!please check the file ora_rpm_install.txt!')
cmd2="cd /dbhome;sh ora_rpm_check.sh>ora_rpm_check.txt"
host.rpm_install(cmd2)
print (' run ora_rpm_check.sh complete!please check the file ora_rpm_check.txt!')

#创建类Pre_root实现oracle安装前的系统参数配置
class Pre_root(object):
    def __init__(self,hostname,username,password):
        self.hostname=hostname
        self.username=username
        self.password=password
    def sftp_put_core(self,localfile,remotefile):
        try:
            socks=(self.hostname,22)
            t=paramiko.Transport((socks))
            t.connect(username=self.username,password=self.password)
            sftp=paramiko.SFTPClient.from_transport(t)
            sftp.put(localfile,remotefile)
        except Exception as e:
            print ('upload file failed:',e)
            t.close()
        finally:
            t.close()
    def remove_old_core(self):
        paramiko.util.log_to_file('remove_old_core.log')
        s=paramiko.SSHClient()
        s.load_system_host_keys()
        s.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        s.connect(self.hostname,22,self.username,self.password)
        stdin,stdout,stderr=s.exec_command('mv /etc/sysctl.conf /etc/sysctl.conf.old;mv /etc/security/limits.conf /etc/security/limits.conf.old;mv /etc/sysctl.conf.new /etc/sysctl.conf;mv /etc/security/limits.conf.new /etc/security/limits.conf);
        stdout.read()
        s.close()
    def create_ora_user(self,__cmd):
        try:
            paramiko.util.log_to_file('create_ora_user.log')
            s=paramiko.SSHClient()
            s.load_system_host_keys()
            s.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            s.connect(self.hostname,22,self.username,self.password)
            stdin,stdout,stderr=s.exec_command(__cmd)
            stdout.read()
        except Exception as e:
            print ('error:',e)
            s.close()
        finally:
            s.close()

#if __name__=="__main__":
localfile1=r'E:\\oraclepy\\oraclerpm\\sysctl.conf'
localfile2=r'E:\\oraclepy\\oraclerpm\\limits.conf'
remotefile1=r'/etc/sysctl.conf.new'
remotefile2=r'/etc/security/limits.conf.new'
cmd0='sysctl -p'
cmd1='cd /;mkdir -p /oracle/product/11.2.0.4/dbhome_1;mkdir -p /oracle/oraInventory'
cmd2='groupadd -g 1000 oinstall;groupadd -g 1100 dba'
cmd3='useradd -u 1006 -g oinstall -G dba -m -d /dbhome/oracle  oracle'
cmd4='cd /;chown -R oracle:oinstall /oracle;chmod -R 755 /oracle'
host=Pre_root(hostname,username,password)
host.sftp_put_core(localfile1,remotefile1)
print ('upload sysctl.conf successful!')
host.sftp_put_core(localfile2,remotefile2)
print ('upload limits.conf successful!')
host.remove_old_core()
print ('update sysctl.conf/limits.conf successful!')
host.create_ora_user(cmd0)                                           
host.create_ora_user(cmd1)
host.create_ora_user(cmd2)
host.create_ora_user(cmd3)
host.create_ora_user(cmd4)
cmdsetp= "echo 'oracle'|passwd --stdin oracle"  #修改指定用户的密码
host.create_ora_user(cmdsetp)

#创建类Pre_oracle实现oracleprofile文件传输
class Pre_oracle(object):
    def __init__(self,hostname,username,password):
        self.hostname=hostname
        self.username=username
        self.password=password
    def remove_old_oracle_profile(self,__cmd):
        paramiko.util.log_to_file('remove_old_oracle_profile.log')
        s=paramiko.SSHClient()
        s.load_system_host_keys()
        s.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        s.connect(hostname,port,username,password)
        stdin,stdout,stderr=s.exec_command(__cmd);
        stdout.read()
        s.close()
    def sftp_put_oracle_profile(self,localfile,remotefile):
        t=paramiko.Transport(hostname,port)
        t.connect(username=username,password=password)
        sftp=paramiko.SFTPClient.from_transport(t)
        sftp.put(localfile,remotefile)
        t.close()

if __name__=="__main__":
    localfile=r'E:\\oraclepy\\oraclerpm\\.bash_profile'
    remotefile=r'/dbhome/oracle/bash_profile.n'
    cmd='cd /dbhome/oracle;mv .bash_profile .bash_profile.old;mv bash_profile.n .bash_profile'
    host=Pre_oracle(hostname,'oracle','oracle')
    host.sftp_put_oracle_profile(localfile,remotefile)
    host.remove_old_oracle_profile(cmd)

#创建类 Oracle_install_rdbms实现oracle软件安装，监听安装，数据库创建
class Oracle_install_rdbms(object):
    def __init__(self,hostname,username,password):
        self.hostname=hostname
        self.username=username
        self.password=password
    def __get_all_file_in_local_dir(self,local_dir):
        all_files=list()
        files=os.listdir(local_dir)
        for x in files:
            filename=os.path.join(local_dir,x)
            if os.path.isdir(x):
                all_files.extend(self.__get_all_file_in_local_dir(filename))
            else:
                all_files.append(filename)
        return all_files
    def sftp_put_dir(self,local_dir,remote_dir):
        t=paramiko.Transport((self.hostname,22))
        t.connect(username=self.username,password=self.password)
        sftp=paramiko.SFTPClient.from_transport(t)
        if remote_dir[-1]=='/':
            remote_dir=remote_dir[0:-1]
        all_files=self.__get_all_file_in_local_dir(local_dir)
        for x in all_files:
            filename=os.path.split(x)[-1]
            remote_filename=remote_dir+'/'+filename
            print (u'Putting file %s...' % filename)
            sftp.put(x,remote_filename)
        t.close()
    def ora_rdbms(self,__cmd):
        try:
            paramiko.util.log_to_file('paramiko_ora_rdbms_install.log')
            s=paramiko.SSHClient()
            s.load_system_host_keys()
            s.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            s.connect(self.hostname,22,self.username,self.password)
            stdin,stdout,stderr=s.exec_command(__cmd)
            stdout.read()
        except Exception as e:
            print ('error:',e)
            s.close()
        finally:
            s.close()

#执行前需手动设置oracle用户密码为oracle，已解决，利用管道标准输入
if __name__=="__main__":
    localdir=r'E:\\oraclepy\\oracle'
    remotedir=r'/oracle'
    cmd1='cd /oracle;unzip p13390677_112040_Linux-x86-64_1of7.zip;unzip p13390677_112040_Linux-x86-64_2of7.zip;rm -rf *.zip'
    cmd2='cd /oracle/database;./runInstaller -silent -force -noconfig -responseFile /oracle/db_install_nx.rsp -ignorePrereq > /oracle/oracle_install.log'
    cmd3='sh /oracle/oraInventory/orainstRoot.sh>/dbhome/oracle/orainstRoot.txt;sh /oracle/product/11.2.0.4/dbhome_1/root.sh>/dbhome/oracle/root.txt'
    cmd4='/oracle/product/11.2.0.4/dbhome_1/bin/netca -silent -responseFile /oracle/netca_nx.rsp > /oracle/netca_install.log'
    cmd5='/oracle/product/11.2.0.4/dbhome_1/bin/dbca -silent -responseFile /oracle/dbca_nx.rsp > /oracle/dbca_install.log'
    host=Oracle_install_rdbms(hostname,'oracle','oracle')
    host.sftp_put_dir(localdir,remotedir)
    host.ora_rdbms(cmd1)
    host.ora_rdbms(cmd2)
    rootuser=Oracle_install_rdbms(hostname,username,password)
    rootuser.ora_rdbms(cmd3)
    #host.ora_rdbms(cmd4)
    #host.ora_rdbms(cmd5)

'''
以下具体数值和参数以实际情况为准
1、修改最大连接数：
sql> show parameter processes;
sql> alter system set processes=2000 scope = spfile;
2、修改用户密码用不过期：
SQL> select * from dba_profiles s where s.profile='DEFAULT' and resource_name='PASSWORD_LIFE_TIME';
PROFILE          RESOURCE_NAME   RESOURCE
------------------------------ -------------------------------- --------
LIMIT
----------------------------------------
DEFAULT          PASSWORD_LIFE_TIME  PASSWORD
180
SQL> alter profile default limit password_life_time unlimited;
Profile altered.
3、设置数据库为自动内存管理模式：
    修改数据库为自动内存管理模式：
SQL> alter system set memory_target=10240M scope=spfile;            //物理内存的60%左右。
System altered.
SQL> alter system set memory_max_target=10240M scope=spfile;    //物理内存的60%左右。
System altered.
SQL> alter system set sga_target=0 scope=spfile;
System altered.
SQL> alter system set sga_max_size=7168M scope=spfile;               //实例内存的70%左右，即memory_max_target*70%，也即物理内存*60%*70%。
System altered.
SQL> alter system set pga_aggregate_target=0 scope=spfile;
System altered.
SQL> alter system set pre_page_sga=FALSE scope=spfile;
System altered.
4、修改redo log组以及大小：--为防止日志频繁切换，引起数据库性能低下问题。
5、修改日志模式为归档模式：
SQL> alter database archivelog;
Database altered.
SQL> alter system set log_archive_format='xtwldb_%t_%s_%r.log' scope=spfile;
System altered.
SQL> alter system set log_archive_dest1='location=/home/oracle/app/archlog' scope=spfile;
System altered.
6.打补丁
'''

Putting file dbca_nx.rsp...
Putting file dbca_nx.rsp.bak...
Putting file db_install_nx.rsp...
Putting file netca_nx.rsp...
Putting file p13390677_112040_Linux-x86-64_1of7.zip...
Putting file p13390677_112040_Linux-x86-64_2of7.zip...
Putting file p20760982_112040_Linux-x86-64.zip...
Putting file p6880880_112000_Linux-x86-64.zip...


"\n以下具体数值和参数以实际情况为准\n1、修改最大连接数：\nsql> show parameter processes;\nsql> alter system set processes=2000 scope = spfile;\n2、修改用户密码用不过期：\nSQL> select * from dba_profiles s where s.profile='DEFAULT' and resource_name='PASSWORD_LIFE_TIME';\nPROFILE          RESOURCE_NAME   RESOURCE\n------------------------------ -------------------------------- --------\nLIMIT\n----------------------------------------\nDEFAULT          PASSWORD_LIFE_TIME  PASSWORD\n180\nSQL> alter profile default limit password_life_time unlimited;\nProfile altered.\n3、设置数据库为自动内存管理模式：\n    修改数据库为自动内存管理模式：\nSQL> alter system set memory_target=10240M scope=spfile;            //物理内存的60%左右。\nSystem altered.\nSQL> alter system set memory_max_target=10240M scope=spfile;    //物理内存的60%左右。\nSystem altered.\nSQL> alter system set sga_target=0 scope=spfile;\nSystem altered.\nSQL> alter system set sga_max_size=7168M scope=spfile;               //实例内存的70%左右，即memory_max_target*70%，也即物理内存*60%*70%。\nSystem altered.\nSQL> alter system set p

### 脚本结束后1、修改最大连接数：2、修改用户密码用不过期：3、设置数据库为自动内存管理模式：4、修改redo log组以及大小：5、修改日志模式为归档模式：6、打补丁

In [10]:
rootuser=Oracle_install_rdbms(hostname,username,password)
rootuser.ora_rdbms(cmd3)